In [1]:
import wandb

run = wandb.init(project="vd_model_training")

# fetch the artifact and download
artifact = run.use_artifact('vd_model_fold_1:latest')
artifact_dir = artifact.download()

run.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: soniamatei. Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact vd_model_fold_1:latest, 471.40MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:3.0


In [2]:
import sys
sys.path.insert(0, '/home/sonia2oo2soia/projects/Thesis-Project/Thesis-Project')  

from dotenv import load_dotenv
load_dotenv('/home/sonia2oo2soia/projects/Thesis-Project/Thesis-Project/.env')

True

In [3]:
import torch
import os

from model.violence_detection_model import ViolenceDetectionModel

device = "cuda" if torch.cuda.is_available() else "cpu"
torch.cuda.empty_cache()
torch.cuda.synchronize()
print("device: {}".format(device))

model_path = os.path.join(artifact_dir, 'vd_model.pth')

vd_model = ViolenceDetectionModel(json_file="model_settings.json").to(device)
vd_model.load_state_dict(torch.load(model_path))
vd_model.eval()

device: cuda


/home/sonia2oo2soia/projects/Thesis-Project/Thesis-Project/venv/lib/python3.12/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


ViolenceDetectionModel(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTSdpaAttention(
            (attention): ViTSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_featu

In [61]:
from torch.utils.data import DataLoader
from torch.nn import BCELoss
from matplotlib import pyplot as plt, animation
from sklearn.metrics import accuracy_score

from data_handling.data_augmentation import VideoTransform
from data_handling.video_dataset import VideoDataset
from utils import collate_fn_pad

hf_transforms = VideoTransform(dataset="HockeyFights", json_file="augmentation_values.json")
hf_dataset = VideoDataset(dataset="HockeyAngry", transformations=hf_transforms)
criterion = BCELoss()

for _ in range(10):
    cummulative_loss = 0
    cummulative_acc = 0

    # loop for interchanging the datasets
    for _ in range(2):
        hf_dataset.flag = not hf_dataset.flag
        dataloader = DataLoader(hf_dataset, batch_size=24, collate_fn=collate_fn_pad, shuffle=True)
        
        for batch in dataloader:
            video, label, lenght = batch
            video, label, lenght = video.to(device), label.to(device), lenght.to(device)
            output = vd_model(video, lenght)
            
            loss = criterion(output, label)
            cummulative_loss += loss.item()

            correct = (label == (output > 0.5).float()).sum()
            cummulative_acc += correct.item()

    print(cummulative_acc)
    print(f"Accuracy: {cummulative_acc / 37 *100}, Loss: {cummulative_loss/2}")


6
Accuracy: 16.216216216216218, Loss: 0.5669353306293488
6
Accuracy: 16.216216216216218, Loss: 0.5403800904750824
6
Accuracy: 16.216216216216218, Loss: 0.5568732619285583
6
Accuracy: 16.216216216216218, Loss: 0.6398013234138489
6
Accuracy: 16.216216216216218, Loss: 0.527237556874752
5
Accuracy: 13.513513513513514, Loss: 0.5714094564318657
6
Accuracy: 16.216216216216218, Loss: 0.5504493415355682
6
Accuracy: 16.216216216216218, Loss: 0.4610436037182808
6
Accuracy: 16.216216216216218, Loss: 0.4614778235554695
6
Accuracy: 16.216216216216218, Loss: 0.519511915743351


In [79]:
from torch.utils.data import DataLoader
from torch.nn import BCELoss
from matplotlib import pyplot as plt, animation
from sklearn.metrics import accuracy_score

from data_handling.data_augmentation import VideoTransform
from data_handling.video_dataset import VideoDataset
from utils import collate_fn_pad

hf_transforms = VideoTransform(dataset="HockeyFights", json_file="augmentation_values.json")
hf_dataset = VideoDataset(dataset="HockeyAngry", transformations=hf_transforms)
dataloader = DataLoader(hf_dataset, batch_size=1, collate_fn=collate_fn_pad, shuffle=True)
criterion = BCELoss()

for batch in dataloader:
    video, label, lenght = batch
    # video.unsqueeze(0) # for batch dimension needed
    video, label, lenght = video.to(device), label.to(device), lenght.to(device)
    output = vd_model(video, lenght)
    
    loss = criterion(output, label)

    print(label, output, loss)

    fig, ax = plt.subplots(figsize=(3, 3))
    frames = batch[0][0].permute(0, 2, 3, 1)

    # define an animation to play th video
    def animate(frame):
        ax.clear()
        ax.imshow(frame.float().clamp(0, 1))
        ax.axis("off")

    ani = animation.FuncAnimation(fig, animate, frames=frames, interval=41)
    ani.save(f'animation.gif', writer='imagemagick')
    plt.close(fig)

    break


MovieWriter imagemagick unavailable; using Pillow instead.


tensor([1.], device='cuda:0') tensor([0.8337], device='cuda:0', grad_fn=<SqueezeBackward1>) tensor(0.1819, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
